In [2]:
# baseline.ipynb

import os, random, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

# ----------------------------
# 1. Reproducibility
# ----------------------------
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# ----------------------------
# 2. Model: 2-layer GCN
# ----------------------------
class GCN(nn.Module):
    def __init__(self, in_ch, hid_ch, out_ch, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_ch, hid_ch)
        self.conv2 = GCNConv(hid_ch, out_ch)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x  # raw logits

# ----------------------------
# 3. Train + Evaluate (one run)
# ----------------------------
def train_one_run(seed=0, dataset_name="Cora",
                  hidden=64, dropout=0.5,
                  weight_decay=5e-4, lr=0.01,
                  max_epochs=200, patience=50, verbose=False):
    set_seed(seed)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    dataset = Planetoid(root=os.path.join("data", dataset_name), name=dataset_name)
    data = dataset[0].to(device)

    model = GCN(dataset.num_node_features, hidden, dataset.num_classes, dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()   # <-- no label smoothing here

    best_val, best_test, best_epoch = -1, 0, -1
    patience_ctr = 0

    for epoch in range(1, max_epochs+1):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # evaluation
        model.eval()
        with torch.no_grad():
            pred = out.argmax(dim=1)
            train_acc = (pred[data.train_mask] == data.y[data.train_mask]).float().mean().item()
            val_acc   = (pred[data.val_mask]   == data.y[data.val_mask]).float().mean().item()
            test_acc  = (pred[data.test_mask]  == data.y[data.test_mask]).float().mean().item()

        if val_acc > best_val:
            best_val, best_test, best_epoch = val_acc, test_acc, epoch
            patience_ctr = 0
        else:
            patience_ctr += 1

        if verbose and epoch % 50 == 0:
            print(f"Epoch {epoch} | Loss {loss:.4f} | Val {val_acc:.3f} | Test {test_acc:.3f}")

        if patience_ctr >= patience:
            break

    return best_val, best_test

# ----------------------------
# 4. Run across seeds
# ----------------------------
seeds = [0,1,2,3,4]
results = []

for s in seeds:
    val, test = train_one_run(seed=s, verbose=False)
    results.append(test)
    print(f"Seed {s}: Test Acc = {test:.4f}")

# ----------------------------
# 5. Summary
# ----------------------------
mean_acc = np.mean(results)
std_acc  = np.std(results)

print(f"\nBaseline GCN on Cora ({len(seeds)} seeds)")
print(f"Test Accuracy = {mean_acc:.4f} ± {std_acc:.4f}")

# Save as DataFrame for plotting
df = pd.DataFrame({
    "Name": ["Baseline"] * len(results),
    "TestAcc": results
})

Seed 0: Test Acc = 0.7960
Seed 1: Test Acc = 0.7570
Seed 2: Test Acc = 0.7810
Seed 3: Test Acc = 0.7850
Seed 4: Test Acc = 0.7800

Baseline GCN on Cora (5 seeds)
Test Accuracy = 0.7798 ± 0.0127
